In [ ]:
import socket
import _thread
import re
import sys
server = socket.socket()
server.bind(('127.0.0.1',8000))
server.listen(3)
conn, addr = server.accept()
data = True
while data :
    data = conn.recv(1024)
    msg = raw_input()
    if msq=="any code you mean to exit": break
    conn.sendall(msg)
conn.close()
server.close()



In [21]:
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('192.168.20.218', 8005))
server.listen(3)
conn, addr = server.accept()
data = conn.recv(1024)
print(data)
# 假装已经实现了getHost
host, port = getHost(data)
target = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
target.connect((host, port))
target.sendall(data)
 
data = target.recv(1024)
print(data)
conn.sendall(data)
 
target.close()
conn.close()
server.close()



b'GET http://8.131.54.75:7500/ HTTP/1.1\r\nHost: 8.131.54.75:7500\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8\r\nProxy-Connection: keep-alive\r\nUpgrade-Insecure-Requests: 1\r\nCookie: wp-settings-1=libraryContent%3Dbrowse%26ampeditor%3Dtinymce%26ampmfold%3Do%26ampposts_list_mode%3Dlist; wp-settings-time-1=1637148760; QZpU_2132_ulastactivity=5f3cPzH9yUjXxPtdvoBWhFZQtqjnEr1t3PB3ozg5FOrIbp5cq0tD; wp-settings-time-4=1626229301\r\nUser-Agent: Mozilla/5.0 (iPhone; CPU iPhone OS 14_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Mobile/15E148 Safari/604.1\r\nAccept-Language: zh-cn\r\nAccept-Encoding: gzip, deflate\r\nConnection: keep-alive\r\n\r\n'


NameError: name 'getHost' is not defined

In [12]:
 

server.close()

In [25]:
def main(_, port=8099):
    myserver = socket.socket()
    myserver.bind(('127.0.0.1', port))
    myserver.listen(1024)
    while True:
        conn, addr = myserver.accept()
        thread_p = threading.Thread(target=thread_proxy, args=(conn, addr))
        thread_p.setDaemon(True)
        thread_p.start()
 
if __name__ == '__main__':
    main(*sys.argv)
    sys.exit(0)



TypeError: main() takes from 1 to 2 positional arguments but 3 were given

In [29]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
    simple-http-proxy ( https://github.com/WengChaoxi/simple-http-proxy )
    ~ ~ ~ ~ ~ ~
    一个简单的http代理
    
    :copyright: (c) 2021 by WengChaoxi.
    :license: MIT, see LICENSE for more details.
"""
from __future__ import print_function

import socket
import select
import time

def debug(tag, msg):
    print('[%s] %s' % (tag, msg))

class HttpRequestPacket(object):
    '''
    HTTP请求包
    '''
    def __init__(self, data):
        self.__parse(data)

    def __parse(self, data):
        '''
        解析一个HTTP请求数据包
        GET http://test.wengcx.top/index.html HTTP/1.1\r\nHost: test.wengcx.top\r\nProxy-Connection: keep-alive\r\nCache-Control: max-age=0\r\n\r\n
        
        参数：data 原始数据
        '''
        i0 = data.find(b'\r\n') # 请求行与请求头的分隔位置
        i1 = data.find(b'\r\n\r\n') # 请求头与请求数据的分隔位置
    
        # 请求行 Request-Line
        self.req_line = data[:i0]
        self.method, self.req_uri, self.version = self.req_line.split() # 请求行由method、request uri、version组成
        
        # 请求头域 Request Header Fields
        self.req_header = data[i0+2:i1]
        self.headers = {}
        for header in self.req_header.split(b'\r\n'):
            k, v = header.split(b': ')
            self.headers[k] = v
        self.host = self.headers.get(b'Host')
        
        # 请求数据
        self.req_data = data[i1+4:]

class SimpleHttpProxy(object):
    '''
    简单的HTTP代理
    客户端(client) <=> 代理端(proxy) <=> 服务端(server)
    '''
    def __init__(self, host='0.0.0.0', port=8080, listen=10, bufsize=8, delay=1):
        '''
        初始化代理套接字，用于与客户端、服务端通信
        参数：host 监听地址，默认0.0.0.0，代表本机任意ipv4地址
        参数：port 监听端口，默认8080
        参数：listen 监听客户端数量，默认10
        参数：bufsize 数据传输缓冲区大小，单位kb，默认8kb
        参数：delay 数据转发延迟，单位ms，默认1ms
        '''
        self.socket_proxy = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket_proxy.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # 将SO_REUSEADDR标记为True, 当socket关闭后，立刻回收该socket的端口
        self.socket_proxy.bind((host, port))
        self.socket_proxy.listen(listen)

        self.socket_recv_bufsize = bufsize*1024
        self.delay = delay/1000.0

        debug('info', 'bind=%s:%s' % (host, port))
        debug('info', 'listen=%s' % listen)
        debug('info', 'bufsize=%skb, delay=%sms' % (bufsize, delay))

    def __del__(self):
        self.socket_proxy.close()
    
    def __connect(self, host, port):
        '''
        解析DNS得到套接字地址并与之建立连接
        参数：host 主机
        参数：port 端口
        返回：与目标主机建立连接的套接字
        '''
        # 解析DNS获取对应协议簇、socket类型、目标地址
        # getaddrinfo -> [(family, sockettype, proto, canonname, target_addr),]
        (family, sockettype, _, _, target_addr) = socket.getaddrinfo(host, port)[0]
        
        tmp_socket = socket.socket(family, sockettype)
        tmp_socket.setblocking(0)
        tmp_socket.settimeout(5)
        tmp_socket.connect(target_addr)
        return tmp_socket
         
    def __proxy(self, socket_client):
        '''
        代理核心程序
        参数：socket_client 代理端与客户端之间建立的套接字
        '''
        # 接收客户端请求数据
        req_data = socket_client.recv(self.socket_recv_bufsize)
        if req_data == b'':
            return

        # 解析http请求数据
        http_packet = HttpRequestPacket(req_data)

        # 获取服务端host、port
        if b':' in http_packet.host:
            server_host, server_port = http_packet.host.split(b':')
        else:
            server_host, server_port = http_packet.host, 80

        # 修正http请求数据
        tmp = b'%s//%s' % (http_packet.req_uri.split(b'//')[0], http_packet.host)
        req_data = req_data.replace(tmp, b'')

        # HTTP
        if http_packet.method in [b'GET', b'POST', b'PUT', b'DELETE', b'HEAD']:
            socket_server = self.__connect(server_host, server_port) # 建立连接
            socket_server.send(req_data) # 将客户端请求数据发给服务端

        # HTTPS，会先通过CONNECT方法建立TCP连接
        elif http_packet.method == b'CONNECT':
            socket_server = self.__connect(server_host, server_port) # 建立连接

            success_msg = b'%s %d Connection Established\r\nConnection: close\r\n\r\n'\
                %(http_packet.version, 200)
            socket_client.send(success_msg) # 完成连接，通知客户端
            
            # 客户端得知连接建立，会将真实请求数据发送给代理服务端
            req_data = socket_client.recv(self.socket_recv_bufsize) # 接收客户端真实数据
            socket_server.send(req_data) # 将客户端真实请求数据发给服务端

        # 使用select异步处理，不阻塞
        self.__nonblocking(socket_client, socket_server)

    def __nonblocking(self, socket_client, socket_server):
        '''
        使用select实现异步处理数据
        参数：socket_client 代理端与客户端之间建立的套接字
        参数：socket_server 代理端与服务端之间建立的套接字
        '''
        _rlist = [socket_client, socket_server]
        is_recv = True
        while is_recv:
            try:
                # rlist, wlist, elist = select.select(_rlist, _wlist, _elist, [timeout])
                # 参数1：当列表_rlist中的文件描述符fd状态为readable时，fd将被添加到rlist中
                # 参数2：当列表_wlist中存在文件描述符fd时，fd将被添加到wlist
                # 参数3：当列表_xlist中的文件描述符fd发生错误时，fd将被添加到elist
                # 参数4：超时时间timeout
                #  1) 当timeout==None时，select将一直阻塞，直到监听的文件描述符fd发生变化时返回
                #  2) 当timeout==0时，select不会阻塞，无论文件描述符fd是否有变化，都立刻返回
                #  3) 当timeout>0时，若文件描述符fd无变化，select将被阻塞timeout秒再返回
                rlist, _, elist = select.select(_rlist, [], [], 2)
                if elist:
                    break
                for tmp_socket in rlist:
                    is_recv = True
                    # 接收数据
                    data = tmp_socket.recv(self.socket_recv_bufsize)
                    if data == b'':
                        is_recv = False
                        continue
                    
                    # socket_client状态为readable, 当前接收的数据来自客户端
                    if tmp_socket is socket_client: 
                        socket_server.send(data) # 将客户端请求数据发往服务端
                        # debug('proxy', 'client -> server')

                    # socket_server状态为readable, 当前接收的数据来自服务端
                    elif tmp_socket is socket_server:
                        socket_client.send(data) # 将服务端响应数据发往客户端
                        # debug('proxy', 'client <- server')
                        
                time.sleep(self.delay) # 适当延迟以降低CPU占用
            except Exception as e:
                break

        socket_client.close()
        socket_server.close()

    def client_socket_accept(self):
        '''
        获取已经与代理端建立连接的客户端套接字，如无则阻塞，直到可以获取一个建立连接套接字
        返回：socket_client 代理端与客户端之间建立的套接字
        '''
        socket_client, _ = self.socket_proxy.accept()
        return socket_client

    def handle_client_request(self, socket_client):
        try:
            self.__proxy(socket_client)
        except:
            pass

    def start(self):
        try:
            import _thread as thread # py3
        except ImportError:
            import thread # py2
        while True:
            try:
                # self.handle_client_request(self.client_socket_accept())
                thread.start_new_thread(self.handle_client_request, (self.client_socket_accept(), ))
            except KeyboardInterrupt:
                break

if __name__ == '__main__':
    # 默认参数
    host, port, listen, bufsize, delay = '0.0.0.0', 8080, 10, 8, 1
    
    import sys, getopt
    try:
        opts, _ = getopt.getopt(sys.argv[1:], 'h:p:l:b:d:', ['host=', 'port=', 'listen=', 'bufsize=', 'delay='])        
        for opt, arg in opts:
            if opt in ('-h', '--host'):
                host = arg
            elif opt in ('-p', '--port'):
                port = int(arg)
            elif opt in ('-l', '--listen'):
                listen = int(arg)
            elif opt in ('-b', '--bufsize'):
                bufsize = int(arg)
            elif opt in ('-d', '--delay'):
                delay = float(arg)
    except:
        debug('error', 'read the readme.md first!')
        sys.exit()
    
    # 启动代理
    SimpleHttpProxy(host, port, listen, bufsize, delay).start()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[error] read the readme.md first!
Traceback (most recent call last):
  File "/var/folders/ch/43_ylpr14gj7_x367vf0hm8w0000gn/T/ipykernel_1286/2941418075.py", line 219, in <module>
    opts, _ = getopt.getopt(sys.argv[1:], 'h:p:l:b:d:', ['host=', 'port=', 'listen=', 'bufsize=', 'delay='])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/getopt.py", line 95, in getopt
    opts, args = do_shorts(opts, args[0][1:], shortopts, args[1:])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/getopt.py", line 195, in do_shorts
    if short_has_arg(opt, shortopts):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/getopt.py", line 211, in short_has_arg
    raise GetoptError(_('option -%s not recognized') % opt, opt)
getopt.GetoptError: option -f not recognized

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/py

TypeError: object of type 'NoneType' has no len()

In [346]:
a=input()
b=""
for i in range(len(a)):
    k=a[i]
    if (a[i]=="/"):
        k=chr(92)+a[i]
        
    b=b+k
data={}
print(b)
#b=repr(b)
#data["a"]=b

#data["a"]=eval(repr(data["a"]).replace('\\\\', '\\')) 

import json

jsonData = '{"qrcode":"'+b+'"}';
print(type(jsonData))
text = json.loads(jsonData)
print(text)





https://pass.cppu.net/tpass/appLogin?uuid=a0e1cc41-3780-4993-90c8-494e80bef6d6&service=https://one.cppu.net/tp_up/
https:\/\/pass.cppu.net\/tpass\/appLogin?uuid=a0e1cc41-3780-4993-90c8-494e80bef6d6&service=https:\/\/one.cppu.net\/tp_up\/
<class 'str'>
{'qrcode': 'https://pass.cppu.net/tpass/appLogin?uuid=a0e1cc41-3780-4993-90c8-494e80bef6d6&service=https://one.cppu.net/tp_up/'}


In [248]:
import re
b=input()
a=b
b=re.findall(r'uuid=(.*)//one.cppu.net',b)
root_dir = 'https:\\'
img_dir1 = '\\'
img_dir2 = 'pass.cppu.net\\'
img_dir3 = ''
img_dir4 = 'tpass\\'
img_dir5 = 'appLogin?uuid='+b[0]+"\\"
img_dir6 = '\\'
img_dir7 = 'one.cppu.net\\'
img_dir8 = ''
img_dir9 = 'tp_up\\'
img_dir10 = ''
img = os.path.join(root_dir, img_dir1, img_dir2,img_dir3,img_dir4,img_dir5,img_dir6,img_dir7,img_dir8,img_dir9,img_dir10)
print(img)

data = {"qrcode":"%s",img}




SyntaxError: ':' expected after dictionary key (2963799283.py, line 19)

In [265]:
print(chr(92))

\


In [345]:
import requests
import json
url = "https://app.cppu.net:8303/cas/client"
# 设置消息头
headers = {
    "Appnest_sessionid": "0f9e8597-230c-49cd-9f36-c36a6b8f9158",
    "Accept": "*/*",
    "Appnest_language": "0",
    "Appnest_version": "1.0",
    "Content-Type":"application/json",
    "Accept-Language": "zh-Hans-CN;q=1, en-CN;q=0.9, ko-CN;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Content-Length": "177",
    "User-Agent": "AppNest-iOS",
    "Appnest_method": "appnest.client.basic.qrcodeconfirm",
    "Connection": "close"
    }
# 设置消息体
data = {"qrcode":"https://pass.cppu.net/tpass/appLogin?uuid=6cc73b7f-ab0b-44ce-af96-56c7d53662bd&service=https://one.cppu.net/tp_up/"}
# 获取相应
response=requests.post(url,headers=headers,data=json.dumps(data))
print("Status code:",response.status_code)
print(response.text)
# 解析相应
info=response.json()

print(data)



Status code: 200
{"msg":"无法识别的二维码","result":1035}
{'qrcode': 'https://pass.cppu.net/tpass/appLogin?uuid=6cc73b7f-ab0b-44ce-af96-56c7d53662bd&service=https://one.cppu.net/tp_up/'}


In [318]:
#b=u"https://pass.cppu.net/tpass/appLogin?uuid=57fe1328-1df3-44be-a2eb-9a067a5ed87b&service=https://one.cppu.net/tp_up/"
b="https://pass.cppu.net"+"\\"+"tpa"

#b = eval(repr(b).replace('/', chr(92)+"/")) 
#print(b)

#data = {"qrcode":eval(repr(b).replace('/', chr(92)+"/"))}
data={"s": "https://test\/test"}
print(json.dumps(data))
#我想要的结果是{"s": "https://test\/test"}

{"s": "https://test\\/test"}


In [236]:


b=re.findall(r'uuid=(.*)//one.cppu.net',b)
print(b)

['7e8e5831-ef48-400d-80ca-92428806127d&service=https:']


In [319]:
json.dumps(obj, skipkeys=False）

SyntaxError: invalid character '）' (U+FF09) (2049928736.py, line 1)

In [ ]:
https:\/\/pass.cppu.net\/tpass\/appLogin?uuid=d3f87aa2-2aac-4dd2-a6f6-3aa57476ff86&service=https:\/\/one.cppu.net\/tp_up\/"}

In [343]:
data = {"qrcode":"https://pass.cppu.net/tpass/appLogin?uuid=6cc73b7f-ab0b-44ce-af96-56c7d53662bd&service=https://one.cppu.net/tp_up/"}
data=json.dumps(data)

print(data)

{"qrcode": "https://pass.cppu.net/tpass/appLogin?uuid=6cc73b7f-ab0b-44ce-af96-56c7d53662bd&service=https://one.cppu.net/tp_up/"}


In [ ]:
import json

jsonData = '{"a":1,"b":2,"c":3,"d":4,"e":5}';

text = json.loads(jsonData)
print(text)

In [347]:
POST /cas/client HTTP/1.1
Host: app.cppu.net:8303
Content-Type: application/json
Accept: */*
Appnest_language: 0
Appnest_version: 1.0
Appnest_sessionid: 0f9e8597-230c-49cd-9f36-c36a6b8f9158
Accept-Language: zh-Hans-CN;q=1, en-CN;q=0.9, ko-CN;q=0.8
Accept-Encoding: gzip, deflate
Content-Length: 177
User-Agent: AppNest-iOS
Appnest_method: appnest.client.basic.qrcodeconfirm
Connection: close

{"param":"https://pass.cppu.net/tpass/appLogin?uuid=a0e1cc41-3780-4993-90c8-494e80bef6d6&service=https://one.cppu.net/tp_up/&token=appnest_0f9e8597-230c-49cd-9f36-c36a6b8f9158"}

SyntaxError: invalid decimal literal (366050686.py, line 7)

In [8]:
import requests
import time
ahs=input()

session = requests.session()

burp0_url = "https://app.cppu.net:8303/cas/client"
burp0_headers = {"Content-Type": "application/json", "Accept": "*/*", "Appnest_language": "0", "Appnest_version": "1.0", "Appnest_sessionid": "d0a7d3ce-d6aa-41c5-9f53-0f10cdaee71c", "Accept-Language": "zh-Hans-CN;q=1, en-CN;q=0.9, ko-CN;q=0.8", "Accept-Encoding": "gzip, deflate", "User-Agent": "AppNest-iOS", "Appnest_method": "appnest.client.basic.qrcoderequest", "Connection": "close"}
burp0_json={"qrcode": ahs}

aab=session.post(burp0_url, headers=burp0_headers, json=burp0_json).text

aabb=re.findall(r'param\":\"(.*)\",\"result\":0,\"type\":\"0',aab)

time.sleep(4)
session = requests.session()

burp0_url = "https://app.cppu.net:8303/cas/client"
burp0_headers = {"Content-Type": "application/json", "Accept": "*/*", "Appnest_language": "0", "Appnest_version": "1.0", "Appnest_sessionid": "d0a7d3ce-d6aa-41c5-9f53-0f10cdaee71c", "Accept-Language": "zh-Hans-CN;q=1, en-CN;q=0.9, ko-CN;q=0.8", "Accept-Encoding": "gzip, deflate", "User-Agent": "AppNest-iOS", "Appnest_method": "appnest.client.basic.qrcodeconfirm", "Connection": "close"}
burp0_json={"param": aabb[0]}

print(session.post(burp0_url, headers=burp0_headers, json=burp0_json).text)

https://pass.cppu.net/tpass/appLogin?uuid=889d2ad2-6db8-4176-aaee-ca93d28d5d3c&service=https://one.cppu.net/tp_up/
{"msg":"成功！","result":0}


In [7]:
#模拟登录app
import requests
import time
import re
session = requests.session()

burp0_url = "https://app.cppu.net:8303/cas/client"
burp0_headers = {"Appnest_Version": "1.1", "Content-Type": "application/json", "Accept": "*/*", "Connection": "close", "Appnest_Method": "appnest.client.basic.login", "Accept-Language": "zh-Hans-CN;q=1, en-CN;q=0.9, ko-CN;q=0.8", "User-Agent": "AppNest-iOS", "Accept-Encoding": "gzip, deflate", "Appnest_Language": "0"}
burp0_json={"clientid": "com.cppu.zhydpt", "clientversion": "5.6.9", "ecid": "jcdx", "handset_id": "e645ab9486e248e4a9af5513f8d7dba2", "hsetmodel": "iPhone11,2", "hsetname": "Apple", "imsi": "e645ab9486e248e4a9af5513f8d7dba2", "ispad": "0", "loginid": "2019160053", "logintype": 0, "network": "wifi", "ostype": "ios", "osversion": "14.3", "password": "9f98ca109cbc417e0bd1aed356c84a59", "serverip": "app.cppu.net", "serverport": "8303", "wifimac": "02:00:00:00:00:00"}
dengluapp=session.post(burp0_url, headers=burp0_headers, json=burp0_json).text

try:
    print(re.findall(r'\"username\":\"(.*)\",\"us',dengluapp)[0]+",登录成功")
except:
    print("登录失败")



登录失败


In [17]:
url1="https://one.cppu.net"
params=""
response1 = session.get(url1,params=params,headers=burp0_headers)

In [4]:
#模拟扫码登录
import requests
import time
ahs=input()

session = requests.session()

burp0_url = "https://app.cppu.net:8303/cas/client"
burp0_headers = {"Content-Type": "application/json", "Accept": "*/*", "Appnest_language": "0", "Appnest_version": "1.0", "Appnest_sessionid": re.findall(r'\"sessionid\":\"(.*)\"\,\"token\":',dengluapp)[0], "Accept-Language": "zh-Hans-CN;q=1, en-CN;q=0.9, ko-CN;q=0.8", "Accept-Encoding": "gzip, deflate", "User-Agent": "AppNest-iOS", "Appnest_method": "appnest.client.basic.qrcoderequest", "Connection": "close"}
burp0_json={"qrcode": ahs}

aab=session.post(burp0_url, headers=burp0_headers, json=burp0_json).text

aabb=re.findall(r'param\":\"(.*)\",\"result\":0,\"type\":\"0',aab)

time.sleep(4)
session = requests.session()

burp0_url = "https://app.cppu.net:8303/cas/client"
burp0_headers = {"Content-Type": "application/json", "Accept": "*/*", "Appnest_language": "0", "Appnest_version": "1.0", "Appnest_sessionid": re.findall(r'\"sessionid\":\"(.*)\"\,\"token\":',dengluapp)[0], "Accept-Language": "zh-Hans-CN;q=1, en-CN;q=0.9, ko-CN;q=0.8", "Accept-Encoding": "gzip, deflate", "User-Agent": "AppNest-iOS", "Appnest_method": "appnest.client.basic.qrcodeconfirm", "Connection": "close"}
burp0_json={"param": aabb[0]}

print(session.post(burp0_url, headers=burp0_headers, json=burp0_json).text)

https://pass.cppu.net/tpass/appLogin?uuid=07fbfd84-124a-47aa…95-efeaeb6e4f9c&service=https://one.cppu.net/tp_up/view?m=up
{"msg":"成功！","result":0}


In [14]:
a={"abilitylist":{"avmeetingability":0,"calendarability":0,"contactability":1,"imability":1,"mailability":0,"mamability":1,"mcmability":1,"mdmability":0,"momentsability":0,"noticeability":1,"remotedesktopability":0,"scheduleability":1,"subscribeability":0},"bindphonenumber":"","bindstatus":0,"bindtype":1,"defaultpage":"firstPage","firstLogin":"false","isallowmodifypwd":1,"latestclientversion":"","locationstatus":0,"logoinfos":[],"msg":"成功！","orgUuid":"a69514d7-c8f1-4842-9db0-3ae0259de803","pageparams":[{"alias":"firstPage","enginetype":0,"hiddenbackbutton":0,"hiddenclosebutton":0,"hiddenstatusbutton":0,"hiddentitlebutton":0,"htmlname":"","jssdkflag":0,"name":"首页","param":"","scheme":"","title":"首页","type":"firstPage"},{"alias":"message","enginetype":0,"hiddenbackbutton":0,"hiddenclosebutton":0,"hiddenstatusbutton":0,"hiddentitlebutton":0,"htmlname":"","jssdkflag":0,"name":"消息","param":"","scheme":"","title":"消息","type":"message"},{"alias":"worktable","enginetype":0,"hiddenbackbutton":0,"hiddenclosebutton":0,"hiddenstatusbutton":0,"hiddentitlebutton":0,"htmlname":"","jssdkflag":0,"name":"校园工作台","param":"","scheme":"","title":"校园工作台","type":"worktable"},{"alias":"contact","enginetype":0,"hiddenbackbutton":0,"hiddenclosebutton":0,"hiddenstatusbutton":0,"hiddentitlebutton":0,"htmlname":"","jssdkflag":0,"name":"校园通讯录","param":"","scheme":"","title":"校园通讯录","type":"contact"},{"alias":"more","enginetype":0,"hiddenbackbutton":0,"hiddenclosebutton":0,"hiddenstatusbutton":0,"hiddentitlebutton":0,"htmlname":"","jssdkflag":0,"name":"我的","param":"","scheme":"","title":"我的","type":"more"}],"policies":{"adminmode":1,"adminsession":60,"adminsessionflag":1,"allowchangeheadportrait":1,"allowsecretchat":1,"appcleaninvalidday":15,"cdnhost":"","channelcleaninvalidday":30,"clientautologinvalidtime":72,"clientqrcodeflag":1,"collectionflag":1,"groupchatvote":1,"groupflag":1,"grouptaskflag":1,"hidecircleapp":1,"hideclouddiskapp":1,"hideconcatapp":1,"hideemailapp":1,"hidescheduleapp":1,"hideserverconfig":1,"hidewatermark":0,"imgrouplimit":250,"imlocalmsgsavetime":-1,"imsendfilesize":512,"isallowmodifypwd":1,"isopencdn":0,"mdmconfigurl":"","mdmstatus":0,"meetingclientid":"0","meetingclientsecretkey":"0","meetingmax":30,"meetingmaxusers":30,"meetingmaxvideos":6,"meetingorgcode":"0","modifypwdnow":0,"momentsanonymous":0,"msgrevocationtime":5,"netdiskfilemaxlimit":512,"noticeattachsize":10,"noticecleaninvalidday":30,"pconlineflag":1,"personalflag":0,"photovideoflag":1,"positionleveldiff":1,"readedflag":1,"remotedesktopopenid":"","remotedesktopopenkey":"","secretchatdestructiontime":30,"secretchatflag":1,"sensitivewords":"3e0d73f2-30f7-441d-a198-a191fdbb5442","vcardflag":1,"verificationinterval":-1,"verificationtype":0,"watermarkcontent":"username@loginid@appname","watermarkscope":[]},"releasenotes":"","result":0,"serveraddress":{"avurl":"","imurl":"app.cppu.net:8305","mailurl":"app.cppu.net:8311","pnstcpurl":"app.cppu.net:8302","pnsurl":"app.cppu.net:8301"},"sessionid":"e189df09-e431-4679-af9a-0e2df28c80fa","token":"appnest_e189df09-e431-4679-af9a-0e2df28c80fa","tokentimeout":259200,"updateflag":"0","updateurl":"","userinfo":{"email":"","imaccount":"254559","loginid":"2019160053","nickname":"杨群","onwerdepartments":[{"departmentid":"2b9333b5-42be-0271-4df8-abc56b674022","departmentname":"智慧警务四队_s","departmenttype":0}],"orgname":"中国人民警察大学","photoid":"","positionlevel":0,"signature":"","skey":"02040a537ee446d82a7d948b5d9a1add","username":"魏志豪","useruuid":"71142c62-ef7d-4fe6-b3d9-ca7f5f8fcdc5"}}

In [61]:
http://lfwjxy.fysso.chaoxing.com/sso/lfwjxy

<class 'str'>


In [67]:
import re
re.findall(r'\"useruuid\":\"(.*)\"}}',dengluapp)

['71142c62-ef7d-4fe6-b3d9-ca7f5f8fcdc5']

In [38]:
a.keys()


dict_keys(['abilitylist', 'bindphonenumber', 'bindstatus', 'bindtype', 'defaultpage', 'firstLogin', 'isallowmodifypwd', 'latestclientversion', 'locationstatus', 'logoinfos', 'msg', 'orgUuid', 'pageparams', 'policies', 'releasenotes', 'result', 'serveraddress', 'sessionid', 'token', 'tokentimeout', 'updateflag', 'updateurl', 'userinfo'])

In [41]:
a["userinfo"]["username"]

'魏志豪'

In [43]:
a["userinfo"]['loginid']

'2019160053'

In [46]:
a["sessionid"]

'e189df09-e431-4679-af9a-0e2df28c80fa'

In [93]:
re.findall(r'\"sessionid\":\"(.*)\"\,\"token\":',dengluapp)

['c8aed86e-76cd-4bce-903e-0e6272cae24e']

In [79]:
re.findall(r'\"useruuid\":\"(.*)\"}}',dengluapp)[0]

'71142c62-ef7d-4fe6-b3d9-ca7f5f8fcdc5'

In [34]:
var qrcodehtml =  document.getElementById("qrcode").innerHTML;

SyntaxError: invalid syntax (2664094856.py, line 1)